# Google's Landmark Retrieval Challenge

**Goal**: Given over a million images of 15K classes, you need to classify them correctly as they lack labels. Such that given a test image, it should be able to retrieve images that belong to similar class.

[Kaggle Challenge](https://www.kaggle.com/c/landmark-retrieval-challenge)

In [1]:
import pandas as pd
import urllib
import shutil
import cv2
import os
import numpy as np
from keras.preprocessing import image     

C:\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#STEP #1: Download images
train_data = pd.read_csv("index.csv")
test_data = pd.read_csv("test.csv")

#print("Shape of training set = ", train_data.shape)
#print("Shape of testing set = ", test_data.shape)

def download_images(outputpath):
    if not os.path.isdir(outputpath):
        os.makedirs(outputpath, exist_ok=True)
    
    count = 0
    for index, row in train_data.iterrows():
        img_id = row['id']
        img_url = row['url']
        img_title = outputpath + '/' +img_id + '.JPG'
        if index < 20:
            try:
                urllib.request.urlretrieve(img_url,img_title) # saves local copy of images
                count += 1
            except:
                print(img_id + "<-- couldn't retrieve IMAGE for this id!!")
        else:
            print("Downloaded ==> " + str(count) + " files!") 
            return

In [3]:
#download_images( 'train')
#download_images('test')

In [4]:
def load_dataset(inputpath):
    list_of_tensors = []
    for dirpath, dirnames, filenames in os.walk(inputpath):
        #print("Files in this '" + inputpath + "' dir is " + str(len(filenames)))
        for filename in filenames:
            img_path = os.path.join(dirpath,filename)
            img = image.load_img(img_path, target_size=(224, 224)) #rescaling images to 224x224
            x = image.img_to_array(img)
            list_of_tensors.append(np.expand_dims(x, axis=0))
    return np.vstack(list_of_tensors)

train_tensors = load_dataset('train').astype('float32')/255 # rescale to [0,1] instead of [0,255] for each pixel
test_tensors = load_dataset('test').astype('float32')/255

#Train and validation split
(train_tensors, valid_tensors) = train_tensors[5:], train_tensors[:5]

print("Training set: ", train_tensors.shape)
print("Validation set: ", valid_tensors.shape)
print("Testing set: ", test_tensors.shape)

Training set:  (15, 224, 224, 3)
Validation set:  (5, 224, 224, 3)
Testing set:  (20, 224, 224, 3)


In [5]:
# get feature set from ImageNet using Keras model - VGG16
from keras import applications
model = applications.vgg16.VGG16(include_top=False, weights='imagenet', pooling='avg')

# extract the features
train_targets = model.predict(train_tensors)#[0]
valid_targets = model.predict(valid_tensors)#[0]

# convert from Numpy to a list of values
train_targets_arr = np.char.mod('%f', train_targets)
valid_targets_arr = np.char.mod('%f', valid_targets)
#features_dict = {"id": metadata['id'], "features": ','.join(features_arr)}

In [6]:
print(train_targets_arr.shape)
print(valid_targets_arr.shape)
print(train_targets_arr.shape[1])

(15, 512)
(5, 512)
512


In [7]:
#Implementing basic CNN from scratch
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

numOfLabels = (train_targets_arr.shape[1]) # based on the dataset description  --- NOT TOO SURE HERE!!! I thought 15K should be the value

cnn = Sequential()
cnn.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Dropout(0.3))
cnn.add(Flatten())
cnn.add(Dense(500, activation='relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(numOfLabels, activation='softmax'))

cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 64)        0         
__________

In [8]:
cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Step 7: Train the model
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch.hdf5', verbose=1, save_best_only=True)

cnn.fit(train_tensors, train_targets_arr, validation_data=(valid_tensors, valid_targets_arr), epochs=1, batch_size=2, callbacks=[checkpointer], verbose=1)


Train on 15 samples, validate on 5 samples
Epoch 1/1
14/15 [===========================>..] - ETA: 0s - loss: 944.0118 - acc: 0.8571     
Epoch 00001: val_loss improved from inf to 1090.56959, saving model to weights.best.from_scratch.hdf5
15/15 [==============================] - 8s 504ms/step - loss: 947.3920 - acc: 0.8667 - val_loss: 1090.5696 - val_acc: 1.0000


In [10]:
# load the weights that yielded the best validation accuracy
cnn.load_weights('weights.best.from_scratch.hdf5')

In [ ]:
# Step 8: Calculate accuracy
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = score[1] *100

print("Test accuracy = ", accuracy, "%")